Read the CSV file and parse the data.
Create sets for each node type (Artist, Genre, Country).
Parse relationships and create appropriate connections between nodes.

1. Read and Parse the CSV File
We'll use Python's built-in csv module to read the CSV file. Each row in the file will be read and its fields will be extracted.

2. Create Sets for Node Types
Artists: Extract spotify_id, name, followers, and popularity.
Genres: Extract genres from the genres field.
Countries: Extract country ISO2 names from chart_hits.
3. Parse Relationships
Artist-Genres: Create relationships between each artist and their genres.
Artist-Country-Hits: Create relationships between each artist and countries, including the number of hits as a property of the relationship.

In [7]:
import csv
import json
#convert location from ISO-2 to ISO-3
import pycountry

# Function to parse genres
def parse_genres(genres_str):
    # Strip whitespace and check for empty genres
    genres_str = genres_str.strip()
    if genres_str in ["[]", ""]:
        return []

    try:
        # Try replacing single quotes with double quotes and escaping existing double quotes
        formatted_str = genres_str.replace('"', '\\"').replace("'", '"')
        return json.loads(formatted_str)
    except json.JSONDecodeError:
        # Handle the exception if JSON parsing still fails
        return []

# Function to parse chart hits
def parse_chart_hits(chart_hits_str):
    if chart_hits_str:
        hits = [hit.strip("[]'").strip(")").split(" (") for hit in chart_hits_str.split(", ")]
        # interpret the number of hits as an integer
        hits = [(country, int(hits)) for country, hits in hits]
        return hits
    return []

# Function to convert ISO 3166-1 alpha-2 codes to ISO 3166-1 alpha-3 codes
def alpha2_to_alpha3(alpha2_code):
    try:
        return pycountry.countries.get(alpha_2=alpha2_code).alpha_3
    except AttributeError:
        # Return alpha2 code for codes that don't have a corresponding alpha-3 code
        return alpha2_code
    
def str_to_float(string):
    try:
        return float(string)
    except ValueError:
        return 0.0

# Read CSV and process data
with open('datasets/spotify/nodes.csv', 'r', encoding='utf-8') as file:
    reader = csv.DictReader(file)

    artists = set()
    genres = set()
    countries = set()
    artist_genre_relations = set()
    artist_country_relations = set()

    for row in reader:
        # Add artist
        artists.add((row['spotify_id'], row['name'], str_to_float(row['followers']), int(row['popularity'])))

        # Process and add genres
        for genre in parse_genres(row['genres']):
            genres.add(genre)
            artist_genre_relations.add((row['spotify_id'], genre))

        # Process and add countries and hits
        for country, hits in parse_chart_hits(row['chart_hits']):
            country = alpha2_to_alpha3(country)
            countries.add(country)
            artist_country_relations.add((row['spotify_id'], country, hits))

# At this point, we have all the sets filled with data
# We can now proceed to create nodes and relationships in Neo4j


To add the data from edges.csv file, which represents featuring relationships between artists, into a new set, we can follow these steps:

Read the edges.csv file: Similar to how we read the other CSV file, we'll use Python's csv module.

Create a Set for Featurings: This set will hold tuples representing the featuring relationships between two artists, identified by their id_0 and id_1 values from the CSV file.

In [8]:
import csv

# Initialize a set to store featuring relationships
featurings = set()

# Read the edges.csv file
with open('datasets/spotify/edges.csv', 'r', encoding='utf-8') as file:
    reader = csv.reader(file)
    next(reader)  # Skip the header row

    for row in reader:
        if len(row) == 2:  # Ensure the row has exactly two elements
            id_0, id_1 = row
            featurings.add((id_0, id_1))

# At this point, the 'featurings' set contains all the featuring relationships

In [9]:
# print the length of each set
print(f"Artists: {len(artists)}")
print(f"Genres: {len(genres)}")
print(f"Countries: {len(countries)}")
print(f"Artist-Genre relations: {len(artist_genre_relations)}")
print(f"Artist-Country relations: {len(artist_country_relations)}")
print(f"Featurings: {len(featurings)}")

Artists: 156422
Genres: 4826
Countries: 71
Artist-Genre relations: 110022
Artist-Country relations: 79186
Featurings: 300386


At this point, we import neo4j library and write the script for adding nodes and relations

In [12]:
# ONLY RUN THIS CELL ONCE
from neo4j import GraphDatabase

# Define the Neo4j connection details
uri = "bolt://localhost:7687"
username = "neo4j"
password = "password"

BATCH_SIZE = 5000

# Connect to the Neo4j database
driver = GraphDatabase.driver(uri, auth=(username, password))

# Creatinng indexes to speed up queries, only if they don't exist yet
with driver.session() as session:
    try:
        session.run("""
            CREATE INDEX artist_id FOR (a:Artist) ON (a.spotify_id)
        """)
        session.run("""
            CREATE INDEX genre_name FOR (g:Genre) ON (g.name)
        """)
        session.run("""
            CREATE INDEX country_name FOR (c:Country) ON (c.name)
        """)
    except:
        pass

# Function to create nodes and relationships
def create_nodes_and_relationships():
    with driver.session() as session:
        # Create nodes using batched Cypher queries
        for i in range(0, len(artists), BATCH_SIZE):
            batch = list(artists)[i:i+BATCH_SIZE] if (i+BATCH_SIZE) < len(artists) else list(artists)[i:len(artists) - 1]
            session.run("""
                UNWIND $artists AS artist
                CREATE (a:Artist {spotify_id: artist[0]})
                SET a.name = artist[1], a.followers = artist[2], a.popularity = artist[3]
            """, artists=batch)

        for i in range(0, len(genres), BATCH_SIZE):
            batch = list(genres)[i:i+BATCH_SIZE] if (i+BATCH_SIZE) < len(genres) else list(genres)[i:len(genres) - 1]
            session.run("""
                UNWIND $genres AS genre
                CREATE (g:Genre {name: genre})
            """, genres=batch)

        for i in range(0, len(countries), BATCH_SIZE):
            batch = list(countries)[i:i+BATCH_SIZE] if (i+BATCH_SIZE) < len(countries) else list(countries)[i:len(countries) - 1]
            session.run("""
                UNWIND $countries AS country
                CREATE (c:Country {name: country})
            """, countries=batch)

        # Create relationships
        for i in range(0, len(artist_genre_relations), BATCH_SIZE):
            batch = list(artist_genre_relations)[i:i+BATCH_SIZE] if (i+BATCH_SIZE) < len(artist_genre_relations) else list(artist_genre_relations)[i:len(artist_genre_relations) - 1]
            session.run("""
                UNWIND $artist_genre_relations AS rel
                MATCH (a:Artist {spotify_id: rel[0]})
                MATCH (g:Genre {name: rel[1]})
                CREATE (a)-[:HAS_GENRE]->(g)
            """, artist_genre_relations=batch)

        for i in range(0, len(artist_country_relations), BATCH_SIZE):
            batch = list(artist_country_relations)[i:i+BATCH_SIZE] if (i+BATCH_SIZE) < len(artist_country_relations) else list(artist_country_relations)[i:len(artist_country_relations) - 1]
            session.run("""
                UNWIND $artist_country_relations AS rel
                MATCH (a:Artist {spotify_id: rel[0]})
                MATCH (c:Country {name: rel[1]})
                CREATE (a)-[:HAS_HIT {hits: rel[2]}]->(c)
            """, artist_country_relations=batch)

        for i in range(0, len(featurings), BATCH_SIZE):
            batch = list(featurings)[i:i+BATCH_SIZE] if (i+BATCH_SIZE) < len(featurings) else list(featurings)[i:len(featurings) - 1]
            session.run("""
                UNWIND $featurings AS rel
                MATCH (a:Artist {spotify_id: rel[0]}) 
                MATCH (b:Artist {spotify_id: rel[1]})
                CREATE (a)-[:FEATURING]->(b)
            """, featurings=batch)

# Call the function to create nodes and relationships
create_nodes_and_relationships()
